# Experimentos con Mistral 7B Instruct

Este notebook incluye el código necesario para:
1. Subir y preprocesar el dataset ReDIAL (desde `movies_with_mentions.csv`, `train_data.jsonl` y `test_data.jsonl`).
2. Configurar e invocar Mistral 7B Instruct.
3. Implementar estrategias de prompting: zero-shot, few-shot y chain-of-thought (CoT).
4. Calcular métricas de recomendación (Precision@K, Recall@K, NDCG@K, Diversidad y Novedad).
5. Comparar resultados de Mistral con baselines Random y Most Popular.


In [ ]:
# Dependencias
!pip install transformers sentence_transformers scikit-learn numpy pandas tqdm bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import json
import random
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics import ndcg_score
from bert_score import score as bert_score

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [ ]:
from google.colab import files
print("Subir los siguientes archivos de el dataset de Redial:")
print("   • movies_with_mentions.csv")
print("   • train_data.jsonl")
print("   • test_data.jsonl")
uploaded = files.upload()

Subir los siguientes archivos de el dataset de Redial:
   • movies_with_mentions.csv
   • train_data.jsonl
   • test_data.jsonl


Saving movies_with_mentions.csv to movies_with_mentions.csv
Saving test_data.jsonl to test_data.jsonl
Saving train_data.jsonl to train_data.jsonl


In [ ]:
df_movies = pd.read_csv("movies_with_mentions.csv")
print("Ejemplo de movies_with_mentions.csv:")
display(df_movies.head())

# Construir diccionario movieId (string) → movieName
movieid2title = {str(row["movieId"]): row["movieName"] for _, row in df_movies.iterrows()}


import re

def extract_profile_from_jsonl(obj):
    # 1) Likes
    liked_titles = []
    for side in ("respondentQuestions", "initiatorQuestions"):
        if isinstance(obj.get(side), dict):
            for mid, info in obj[side].items():
                if info.get("liked", 0) == 1:
                    title = obj["movieMentions"].get(mid) or movieid2title.get(mid)
                    if title:
                        liked_titles.append(title)

    # 2) Explicit prefs y último turno válido
    explicit_prefs = []
    last_turn = ""

    if isinstance(obj.get("messages"), list):
        user_id = obj.get("initiatorWorkerId")
        # Buscamos hacia atrás el último turno “útil” del usuario
        for turn in reversed(obj["messages"]):
            if turn.get("senderWorkerId") != user_id:
                continue
            text = turn.get("text", "").strip()
            # Consideramos “válido” si hay signo de pregunta o palabras de recomendación
            if "?" in text or re.search(r"\b(recommend|looking for|busco|sugieres)\b", text, re.I):
                last_turn = text
                break

        # Si no encontramos ninguno, como fallback tomamos simplemente el último turno de usuario
        if not last_turn:
            for turn in reversed(obj["messages"]):
                if turn.get("senderWorkerId") == user_id:
                    last_turn = turn.get("text", "").strip()
                    break

        # Extraemos prefs de géneros del último turno real
        gens = re.findall(
            r"(?i)comedy|drama|action|science fiction|horror|romance|documentary",
            last_turn
        )
        explicit_prefs = list({g.lower() for g in gens})

    return liked_titles, explicit_prefs, last_turn

redial_profiles = {}
uid_counter = 0

with open("train_data.jsonl", "r", encoding="utf-8") as f_train:
    for line in tqdm(f_train, desc="Procesando train_data.jsonl"):
        obj = json.loads(line)
        user_id = f"train_{uid_counter}"
        uid_counter += 1

        liked, prefs, last_msg = extract_profile_from_jsonl(obj)
        redial_profiles[user_id] = {
            "liked_movies": liked,
            "explicit_preferences": prefs,
            "last_turn": last_msg
        }

with open("test_data.jsonl", "r", encoding="utf-8") as f_test:
    for line in tqdm(f_test, desc="Procesando test_data.jsonl"):
        obj = json.loads(line)
        user_id = f"test_{uid_counter}"
        uid_counter += 1

        liked, prefs, last_msg = extract_profile_from_jsonl(obj)
        redial_profiles[user_id] = {
            "liked_movies": liked,
            "explicit_preferences": prefs,
            "last_turn": last_msg
        }

print(f"➡️ Total perfiles generados: {len(redial_profiles)}")


all_titles = df_movies["movieName"].tolist()
print(f"Número de títulos únicos (catálogo completo): {len(all_titles)}")


with open("reprocessed_redial.json", "w", encoding="utf-8") as f_out:
    json.dump(redial_profiles, f_out, ensure_ascii=False, indent=2)

print("✅ Preprocesamiento completado. 'reprocessed_redial.json' creado.")

Ejemplo de movies_with_mentions.csv:


,movieId,movieName,nbMentions
0,75796,Headhunter (2009),1
1,75815,Angels in the Outfield (1994),6
2,75822,Eddie and the Cruisers (1983),1
3,75828,Ninja Assassin (2009),5
4,75867,Orgazmo (1997),3


Procesando train_data.jsonl: 10006it [00:00, 12760.84it/s]
Procesando test_data.jsonl: 1342it [00:00, 14016.39it/s]


➡️ Total perfiles generados: 11348
Número de títulos únicos (catálogo completo): 6924
✅ Preprocesamiento completado. 'reprocessed_redial.json' creado.


In [ ]:
def mostrar_ejemplo_jsonl(archivo, num_lineas=3):
    print(f"📄 Mostrando las primeras {num_lineas} líneas de {archivo}:\n")
    with open(archivo, 'r', encoding='utf-8') as f:
        for i, linea in enumerate(f):
            if i >= num_lineas:
                break
            try:
                obj = json.loads(linea)
                print(json.dumps(obj, indent=2, ensure_ascii=False))
                print("-" * 50)
            except json.JSONDecodeError as e:
                print(f"❌ Error en la línea {i+1}: {e}")
                print(linea)


mostrar_ejemplo_jsonl("train_data.jsonl", num_lineas=3)

📄 Mostrando las primeras 3 líneas de train_data.jsonl:

{
  "movieMentions": {
    "203371": "Final Fantasy: The Spirits Within (2001)",
    "84779": "The Triplets of Belleville (2003)",
    "122159": "Mary and Max (2009)",
    "151313": "A Scanner Darkly  (2006)",
    "191602": "Waking Life (2001)",
    "165710": "The Boss Baby (2017)"
  },
  "respondentQuestions": {
    "203371": {
      "suggested": 1,
      "seen": 0,
      "liked": 1
    },
    "84779": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "122159": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "151313": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "191602": {
      "suggested": 0,
      "seen": 1,
      "liked": 1
    },
    "165710": {
      "suggested": 1,
      "seen": 0,
      "liked": 1
    }
  },
  "messages": [
    {
      "timeOffset": 0,
      "text": "Hi there, how are you? I'm looking for movie recommendations",
      "senderWorkerId": 0,
  

In [ ]:
# ---- Baselines (conteo de liked_movies) ----

import random


pop_counts = {}
for profile in redial_profiles.values():
    for title in profile.get("liked_movies", []):
        pop_counts[title] = pop_counts.get(title, 0) + 1


sorted_pop_titles = sorted(pop_counts.keys(), key=lambda x: pop_counts[x], reverse=True)


def baseline_random(all_items, k=10):
    return random.sample(all_items, k)

def baseline_most_popular(k=10):
    return sorted_pop_titles[:k] if sorted_pop_titles else []

# Ej:
user_example = list(redial_profiles.keys())[0]
print("Usuario de prueba:", user_example)
print("Random:", baseline_random(all_titles, 10))
print("Most Popular (según liked):", baseline_most_popular(10))

Usuario de prueba: train_0
Random: ['The Legend of Frosty the Snowman (2005)', 'Hudson Hawk (1991)', 'Frida (2002)', 'Crac (1981)', 'Beauty and the Bestie (2015)', 'The Code  (2001)', 'The Adventures of Tintin  (2011)', 'Frontier (2016)', 'Princess  (2008)', 'Wild Hogs (2007)']
Most Popular (según liked): ['Black Panther (2018)', 'It  (2017)', 'Jumanji  (2017)', 'Get Out (2017)', 'Deadpool  (2016)', 'The Avengers  (2012)', 'Coco  (2017)', 'Moana  (2016)', 'Bridesmaids  (2011)', 'Die Hard (1988)']


In [ ]:
# ---- Métricas de evaluación ----

import numpy as np
from sklearn.metrics import ndcg_score
from sentence_transformers import SentenceTransformer

# 1) Precision@K
def precision_at_k(y_true, y_pred, k):
    """
    y_true: lista de títulos que el usuario marcó como 'liked' (ground truth).
    y_pred: lista de títulos recomendados por el modelo (ordenados).
    """
    pred_k = y_pred[:k]
    relevant = set(y_true)
    return sum(1 for p in pred_k if p in relevant) / k

# 2) Recall@K
def recall_at_k(y_true, y_pred, k):
    pred_k = y_pred[:k]
    relevant = set(y_true)
    if not relevant:
        return 0.0
    return sum(1 for p in pred_k if p in relevant) / len(relevant)

# 3) NDCG@K
def ndcg_at_k(y_true, y_pred, k):
    """
    Construye vectores de relevancia binaria para calcular NDCG:
      - relevance_true[i] = 1 si el ítem i está en y_true, 0 en otro caso.
      - relevance_pred[i] = 1 si el ítem i está en las primeras k de y_pred, 0 en otro caso.
    """
    items = list(set(y_true + y_pred[:k]))
    relevance_true = [1 if item in y_true else 0 for item in items]
    relevance_pred = [1 if item in y_pred[:k] else 0 for item in items]
    # ndcg_score espera arrays 2D
    return ndcg_score([relevance_true], [relevance_pred], k=k)

# 4) Intra-List Diversity (usando embeddings de títulos)
embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
title_embeddings = embed_model.encode(all_titles, convert_to_tensor=True)
title_to_emb = {title: emb for title, emb in zip(all_titles, title_embeddings)}

def intra_list_diversity(y_pred, k):
    """
    Calcula la diversidad promedio 1 - similitud coseno entre cada par de ítems en top-k.
    """
    pred_k = [t for t in y_pred[:k] if t in title_to_emb]
    if len(pred_k) < 2:
        return 0.0

    vecs = [title_to_emb[t].cpu().numpy() for t in pred_k]
    n = len(vecs)
    sum_dist = 0.0
    count = 0
    for i in range(n):
        for j in range(i+1, n):
            # similitud coseno
            sim = np.dot(vecs[i], vecs[j]) / (np.linalg.norm(vecs[i]) * np.linalg.norm(vecs[j]) + 1e-9)
            sum_dist += (1 - sim)
            count += 1

    return sum_dist / count if count > 0 else 0.0

# 5) Novedad (Novelty) basada en pop_counts
popularity_dict = {title: pop_counts.get(title, 0) / len(redial_profiles) for title in all_titles}

def novelty_at_k(y_pred, k):
    """
    Para cada título en top-k, calcula -log2(P(title)) donde P(title) = freq_liked(title) / total_usuarios.
    Luego promedia sobre k.
    """
    pred_k = y_pred[:k]
    nov_vals = []
    for t in pred_k:
        p = popularity_dict.get(t, 1e-9)
        nov_vals.append(-np.log2(p + 1e-9))
    return sum(nov_vals) / k


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
### Para esta parte se requiere token de huggin face !

!pip install transformers sentence_transformers scikit-learn numpy pandas tqdm bert-score huggingface_hub ipywidgets accelerate

from huggingface_hub import notebook_login
notebook_login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.9 MB/s eta 0:00:00


In [ ]:
# ⋆⋆⋆ Cargar modelo Mistral 7B Instruct v0.1 (público) ⋆⋆⋆
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "mistralai/Mistral-7B-instruct-v0.1"


tokenizer_mistral = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True,
)
# Si quieres trabajar con contextos largos:
tokenizer_mistral.model_max_length = 32768
model_mistral = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

def query_mistral(prompt_text):
    inputs = tokenizer_mistral(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)
    outputs = model_mistral.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7
    )
    return tokenizer_mistral.decode(outputs[0], skip_special_tokens=True).strip()

print("✅ Modelo Mistral 7B Instruct v0.2 cargado correctamente.")


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ Modelo Mistral 7B Instruct v0.2 cargado correctamente.


In [ ]:
# ---- Zero-shot prompt ----
def build_zero_shot_prompt(profile):
    liked = profile.get("liked_movies", [])
    prefs = profile.get("explicit_preferences", [])
    last_turn = profile.get("last_turn", "")

    # Construimos el bloque de perfil
    profile_text = []
    if liked:
        profile_text.append("He disfrutado estas películas: " + ", ".join(liked))
    if prefs:
        profile_text.append("Mis géneros favoritos son: " + ", ".join(prefs))
    if not profile_text:
        profile_text = ["No dispongo de información previa sobre mis gustos."]

    # Instrucción clara: generar 10 recomendaciones nuevas
    instruction = (
        "En base a mis gustos anteriores, **propón 10 películas nuevas** que respondan a mi última petición. "
        "Numera cada recomendación del 1 al 10 y añade entre paréntesis el año."
    )

    return (
        "\n".join(profile_text) + "\n\n"
        f"HUMAN: {last_turn}\n"
        f"INSTRUCCIÓN: {instruction}\n"
        "RECOMENDACIONES:"
    )


# ---- Few-shot examples revisados ----
few_shot_examples = [
    # Ejemplo 1: comedia slapstick 80/90
    """HUMAN: I am looking for a slap stick comedy from the 80’s or 90’s.
INSTRUCCIÓN: Propón 5 películas new que encajen con esta descripción, numeradas.
RECOMENDACIONES:
1. Spaceballs (1987)
2. The Naked Gun (1988)
3. Beetlejuice (1988)
4. Home Alone (1990)
5. A Fish Called Wanda (1988)

""",
    # Ejemplo 2: animación dramática
    """HUMAN: Do you have any animated recommendations that are a bit more dramatic? Like A Scanner Darkly for example.
INSTRUCCIÓN: Propón 5 películas new que encajen con esta descripción, numeradas.
RECOMENDACIONES:
1. Waking Life (2001)
2. Mary and Max (2009)
3. Perfect Blue (1997)
4. Grave of the Fireflies (1988)
5. The Iron Giant (1999)

"""
]

# ---- Few-shot prompt builder ----
def build_few_shot_prompt(profile):
    liked = profile.get("liked_movies", [])
    prefs = profile.get("explicit_preferences", [])
    last_turn = profile.get("last_turn", "")

    # Perfil
    profile_text = []
    if liked:
        profile_text.append("He disfrutado estas películas: " + ", ".join(liked))
    if prefs:
        profile_text.append("Mis géneros favoritos son: " + ", ".join(prefs))
    if not profile_text:
        profile_text = ["No dispongo de información previa sobre mis gustos."]

    # Ensamblamos
    prompt = ""
    for ex in few_shot_examples:
        prompt += ex
    prompt += "\n" + "\n".join(profile_text) + "\n\n"
    prompt += f"HUMAN: {last_turn}\n"
    prompt += "INSTRUCCIÓN: Basándote en lo anterior, propón 10 películas nuevas que respondan a mi petición. Numera del 1 al 10.\n"
    prompt += "RECOMENDACIONES:"

    return prompt


# ---- Chain-of-Thought prompt builder ----
def build_cot_prompt(profile):
    liked = profile.get("liked_movies", [])
    prefs = profile.get("explicit_preferences", [])
    last_turn = profile.get("last_turn", "")

    profile_block = []
    if liked:
        profile_block.append(f"Gustos previos: {', '.join(liked)}")
    if prefs:
        profile_block.append(f"Géneros preferidos: {', '.join(prefs)}")
    if not profile_block:
        profile_block = ["No hay datos de gustos previos."]

    # Prompt con ejemplo de razonamiento completo
    return (
        "Eres un asistente experto en cine.\n"
        + "\n".join(profile_block) + "\n"
        "Sigue este razonamiento **interno** (no lo muestres) y al final entrega **solo** las 10 recomendaciones numeradas:\n"
        "1) Identifica los géneros y temas clave.\n"
        "2) Busca candidatas en tu base de conocimientos.\n"
        "3) Selecciona las 10 más adecuadas y ordénalas.\n"
        "4) Para cada una escribe su título y año.\n\n"
        f"HUMAN: {last_turn}\n"
        "RECOMENDACIONES:"
    )

In [ ]:
with open("reprocessed_redial.json", "r", encoding="utf-8") as f:
    redial_profiles = json.load(f)

print(redial_profiles['train_9004'])

{'liked_movies': ['Red Dawn (1984)', 'Apocalypse Now (1979)', "In Harm's Way (1965)", 'Black Hawk Down  (2002)', 'Saving Private Ryan (1998)', 'American Sniper (2014)', 'Dunkirk  (2017)', 'Lone Survivor (2013)', 'Windtalkers (2002)', 'Red Dawn (1984)', 'Apocalypse Now (1979)', "In Harm's Way (1965)", 'Black Hawk Down  (2002)', 'Saving Private Ryan (1998)', 'American Sniper (2014)', 'Dunkirk  (2017)', 'Lone Survivor (2013)', 'Windtalkers (2002)'], 'explicit_preferences': [], 'last_turn': "I'm looking for war movies like @180728 or @88141"}


In [ ]:
from google.colab import drive
from tqdm.notebook import tqdm
import json
import pandas as pd
import random
import os

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Cargar datos y perfiles
df_movies = pd.read_csv("movies_with_mentions.csv")
movieid2title = {str(r["movieId"]): r["movieName"] for _, r in df_movies.iterrows()}

with open("reprocessed_redial.json", "r", encoding="utf-8") as f:
    redial_profiles = json.load(f)

all_titles = df_movies["movieName"].tolist()

results = []
max_users = 60
count = 0

# Muestreamos 75 IDs de usuario al azar (sin reemplazo)
all_user_ids = list(redial_profiles.keys())
random.shuffle(all_user_ids)
selected_ids = all_user_ids[:max_users]

# 4. Carpeta en Drive para resultados
drive_folder = '/content/drive/MyDrive/mistral_results'
os.makedirs(drive_folder, exist_ok=True)

for user_id in tqdm(selected_ids, desc="Inferencia en usuarios"):
    profile = redial_profiles[user_id]
    print(user_id, "&&", profile)
    if count >= max_users:
        break
    count += 1


    # 1) Zero-shot
    try:
        prompt_zero = build_zero_shot_prompt(profile)
        print(prompt_zero)
        resp_zero = query_mistral(prompt_zero)
        lines_zero = [line.strip() for line in resp_zero.split("\n") if line.strip()]
        recs_zero = []
        for line in lines_zero:
            clean = line
            # eliminar numeración inicial como "1. " o "1) "
            if len(clean) > 1 and clean[0].isdigit() and clean[1] in [".", ")"]:
                clean = clean[2:].strip()
            recs_zero.append(clean)
            if len(recs_zero) >= 10:
                break
    except Exception as e:
        print(f"Error en Zero-shot para {user_id}: {e}")
        recs_zero = []

    # 2) Few-shot
    try:
        prompt_few = build_few_shot_prompt(profile)
        print(prompt_few)
        resp_few = query_mistral(prompt_few)
        lines_few = [line.strip() for line in resp_few.split("\n") if line.strip()]
        recs_few = []
        for line in lines_few:
            clean = line
            if len(clean) > 1 and clean[0].isdigit() and clean[1] in [".", ")"]:
                clean = clean[2:].strip()
            recs_few.append(clean)
            if len(recs_few) >= 10:
                break
    except Exception as e:
        print(f"Error en Few-shot para {user_id}: {e}")
        recs_few = []

    # 3) Chain-of-Thought (CoT)
    try:
        prompt_cot = build_cot_prompt(profile)
        print(prompt_cot)
        resp_cot = query_mistral(prompt_cot)
        lines_cot = [line.strip() for line in resp_cot.split("\n") if line.strip()]
        recs_cot = []
        for line in lines_cot:
            clean = line
            if len(clean) > 1 and clean[0].isdigit() and clean[1] in [".", ")"]:
                clean = clean[2:].strip()
            recs_cot.append(clean)
            if len(recs_cot) >= 10:
                break
    except Exception as e:
        print(f"Error en CoT para {user_id}: {e}")
        recs_cot = []

    # 4) Baselines
    recs_rand = baseline_random(all_titles, 10)
    recs_mp   = baseline_most_popular(10)

    # 5) Ground truth
    ground_truth = profile.get("liked_movies", [])


    results.append({
        "user_id": user_id,
        "method": "Mistral_zero",
        "recs": recs_zero,
        "truth": ground_truth
    })
    results.append({
        "user_id": user_id,
        "method": "Mistral_few",
        "recs": recs_few,
        "truth": ground_truth
    })
    results.append({
        "user_id": user_id,
        "method": "Mistral_cot",
        "recs": recs_cot,
        "truth": ground_truth
    })
    results.append({
        "user_id": user_id,
        "method": "Random",
        "recs": recs_rand,
        "truth": ground_truth
    })
    results.append({
        "user_id": user_id,
        "method": "MostPopular",
        "recs": recs_mp,
        "truth": ground_truth
    })

    # Guardado parcial cada 10 usuarios
    if count % 10 == 0:
        partial_df   = pd.DataFrame(results)
        partial_path = os.path.join(drive_folder, f"results_partial_{count}.csv")
        partial_df.to_csv(partial_path, index=False)
        print(f"✅ Guardado parcial: {partial_path}")


# 6. Guardado final en Drive
df_results = pd.DataFrame(results)
final_path = os.path.join(drive_folder, "results_mistral_experiments_subset60.csv")
df_results.to_csv(final_path, index=False)
print(f"🎉 Inferencia completada. CSV final guardado en: {final_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Inferencia en usuarios:   0%|          | 0/60 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


train_4345 && {'liked_movies': ['The Mirror  (2015)', 'The Conjuring 2 (2016)', 'The Conjuring (2013)', 'It  (2017)', 'The Mirror  (2015)', 'The Conjuring 2 (2016)', 'The Conjuring (2013)'], 'explicit_preferences': [], 'last_turn': 'no thanks for the good suggestions good bye'}
He disfrutado estas películas: The Mirror  (2015), The Conjuring 2 (2016), The Conjuring (2013), It  (2017), The Mirror  (2015), The Conjuring 2 (2016), The Conjuring (2013)

HUMAN: no thanks for the good suggestions good bye
INSTRUCCIÓN: En base a mis gustos anteriores, **propón 10 películas nuevas** que respondan a mi última petición. Numera cada recomendación del 1 al 10 y añade entre paréntesis el año.
RECOMENDACIONES:


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

train_490 && {'liked_movies': ['Shutter Island  (2010)', 'Seven  (1995)', 'The Usual Suspects (1995)', 'Fight Club (1999)', 'Prisoners  (2013)', 'Shutter Island  (2010)', 'Seven  (1995)', 'The Usual Suspects (1995)', 'Fight Club (1999)', 'Prisoners  (2013)'], 'explicit_preferences': [], 'last_turn': 'What is that one about ?'}
He disfrutado estas películas: Shutter Island  (2010), Seven  (1995), The Usual Suspects (1995), Fight Club (1999), Prisoners  (2013), Shutter Island  (2010), Seven  (1995), The Usual Suspects (1995), Fight Club (1999), Prisoners  (2013)

HUMAN: What is that one about ?
INSTRUCCIÓN: En base a mis gustos anteriores, **propón 10 películas nuevas** que respondan a mi última petición. Numera cada recomendación del 1 al 10 y añade entre paréntesis el año.
RECOMENDACIONES:


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


HUMAN: I am looking for a slap stick comedy from the 80’s or 90’s.
INSTRUCCIÓN: Propón 5 películas new que encajen con esta descripción, numeradas.
RECOMENDACIONES:
1. Spaceballs (1987)
2. The Naked Gun (1988)
3. Beetlejuice (1988)
4. Home Alone (1990)
5. A Fish Called Wanda (1988)

HUMAN: Do you have any animated recommendations that are a bit more dramatic? Like A Scanner Darkly for example.
INSTRUCCIÓN: Propón 5 películas new que encajen con esta descripción, numeradas.
RECOMENDACIONES:
1. Waking Life (2001)
2. Mary and Max (2009)
3. Perfect Blue (1997)
4. Grave of the Fireflies (1988)
5. The Iron Giant (1999)


He disfrutado estas películas: Shutter Island  (2010), Seven  (1995), The Usual Suspects (1995), Fight Club (1999), Prisoners  (2013), Shutter Island  (2010), Seven  (1995), The Usual Suspects (1995), Fight Club (1999), Prisoners  (2013)

HUMAN: What is that one about ?
INSTRUCCIÓN: Basándote en lo anterior, propón 10 películas nuevas que respondan a mi petición. Numera del 

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Eres un asistente experto en cine.
Gustos previos: Shutter Island  (2010), Seven  (1995), The Usual Suspects (1995), Fight Club (1999), Prisoners  (2013), Shutter Island  (2010), Seven  (1995), The Usual Suspects (1995), Fight Club (1999), Prisoners  (2013)
Sigue este razonamiento **interno** (no lo muestres) y al final entrega **solo** las 10 recomendaciones numeradas:
1) Identifica los géneros y temas clave.
2) Busca candidatas en tu base de conocimientos.
3) Selecciona las 10 más adecuadas y ordénalas.
4) Para cada una escribe su título y año.

HUMAN: What is that one about ?
RECOMENDACIONES:
✅ Guardado parcial: /content/drive/MyDrive/mistral_results/results_partial_60.csv
🎉 Inferencia completada. CSV final guardado en: /content/drive/MyDrive/mistral_results/results_mistral_experiments_subset75.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import ast
import numpy as np

# 1. (Opcional) Montar tu Google Drive en Colab si aún no lo has hecho:
# from google.colab import drive
# drive.mount('/content/drive')

# 2. Ruta al CSV en tu Drive
file_path = '/content/drive/MyDrive/mistral_results/results_mistral_experiments_subset75.csv'

# 3. Carga del DataFrame
df_results = pd.read_csv(file_path)

In [ ]:
import numpy as np

# --- Asegúrate de tener estas columnas en df_results ---
df_results['recs_list']  = df_results['recs'].apply(ast.literal_eval)
df_results['truth_list'] = df_results['truth'].apply(ast.literal_eval)

ks = [20, 15, 10]
metrics = []

for method, group in df_results.groupby("method"):
    # pre‐aloca
    precisions = {k: [] for k in ks}
    recalls    = {k: [] for k in ks}
    ndcgs      = {k: [] for k in ks}
    diversities= {k: [] for k in ks}
    novelties  = {k: [] for k in ks}

    for _, row in group.iterrows():
        y_true = row["truth_list"]
        y_pred = row["recs_list"]

        for k in ks:
            kp = min(k, len(y_pred))             # normalizamos k
            precisions[k].append( precision_at_k(y_true, y_pred, kp) )
            recalls[k].append(    recall_at_k(   y_true, y_pred, kp) )
            ndcgs[k].append(      ndcg_at_k(     y_true, y_pred, kp) )
            diversities[k].append(intra_list_diversity(y_pred, kp))
            novelties[k].append(  novelty_at_k(  y_pred,       kp))

    # empaqueta resultados
    for k in ks:
        metrics.append({
            "method":        method,
            "k":             k,
            f"Precision@{k}": np.mean(precisions[k]),
            f"Recall@{k}":    np.mean(recalls[k]),
            f"NDCG@{k}":      np.mean(ndcgs[k]),
            f"Diversity@{k}": np.mean(diversities[k]),
            f"Novelty@{k}":   np.mean(novelties[k]),
        })

df_metrics = pd.DataFrame(metrics)

# (Opcional) Pivotea para verlo más claro
df_pivot = df_metrics.pivot(index="method", columns="k")
df_pivot = df_pivot[
    [f"Precision@{k}"     for k in ks] +
    [f"Recall@{k}"      for k in ks] +
    [f"NDCG@{k}"      for k in ks] +
    [f"Diversity@{k}" for k in ks] +
    [f"Novelty@{k}" for k in ks]
]

print(df_pivot)

# 5) Guardar en Drive
output_path = '/content/drive/MyDrive/mistral_results/metrics_summary.csv'
df_pivot.to_csv(output_path, index=True, encoding='utf-8')
print(f"✅ Métricas guardadas en: {output_path}")

             Precision@20               Precision@15                \
k                      10  15        20           10        15  20   
method                                                               
Mistral_cot           NaN NaN  0.006667          NaN  0.006667 NaN   
Mistral_few           NaN NaN  0.003333          NaN  0.003333 NaN   
Mistral_zero          NaN NaN  0.006667          NaN  0.006667 NaN   
MostPopular           NaN NaN  0.030000          NaN  0.030000 NaN   
Random                NaN NaN  0.001667          NaN  0.001667 NaN   

             Precision@10         Recall@20  ... Diversity@10 Novelty@20      \
k                      10  15  20        10  ...           20         10  15   
method                                       ...                               
Mistral_cot      0.006667 NaN NaN       NaN  ...          NaN        NaN NaN   
Mistral_few      0.003333 NaN NaN       NaN  ...          NaN        NaN NaN   
Mistral_zero     0.006667 NaN NaN      